## Niryo Ned robot control experiments
In this notebook we explore the use of pyniryo robot arm for various behaviour planning, control and management

### Connet robot
connect to the robot based on the IP address of the robot, you can either connect to the robot through the robots hotspot or connect the robot to a shared network between your computer and the robot

In [60]:
import pyniryo
# if you are connected to the robot through the robot's hotspot this is the 
# default ip address for the hotspot connection
# although we need to explore the possibility of connecting the robot to a shared network
# to computer make network requests while connected to the robot
ROBOT_IP_ADDRESS = "10.10.10.10"

# If the computer is not connected to the same network as the robot it raises a network connection exception
try:
    Robot = pyniryo.NiryoRobot(ROBOT_IP_ADDRESS)
except Exception as e:
    print("You are not connectied to the same network as the robot")


Connected to server (10.10.10.10) on port: 40001



In [2]:
# Disconnect Robot
Robot.close_connection()


Disconnected from robot



###  Initial robot Setup process
- Calibrate the robot
- Check the learning mode
- Get the hardware status of the robot
- Check the tool in use
- Move the robot to  home position

In [61]:
# Calibrate the robot
Robot.calibrate(calibrate_mode=pyniryo.CalibrateMode.AUTO)

When the learning mode is set to false you are not able to manually adjust the robot joints as they become locked to a particular position

In [65]:
# Experiment with adjusting the learnining mode
Robot.set_learning_mode(False)
current_learning_mode = Robot.get_learning_mode()
print(current_learning_mode)

False


In [14]:
# Check the hardware status of the robot
hardware_status = Robot.get_hardware_status()
# the hardware status shows information such as 
# - Temperature
# - Calibration needed or in progress
# - Motor info, names, temperatures and voltages
# - It also includes potential hardware errors
print(hardware_status)

Temp (°C) : 41.0
Hardware version : 1
Connection Up : True
Error Message : ''
Calibration Needed : False
Calibration in progress : False
MOTORS INFOS : Motor1, Motor2, Motor3, Motor4, Motor5, Motor6,
Names : ['joint_1', 'joint_2', 'joint_3', 'joint_4', 'joint_5', 'joint_6', 'Tool']
Types : ['Niryo Stepper', 'Niryo Stepper', 'Niryo Stepper', 'DXL XL-430', 'DXL XL-430', 'DXL XL-320', 'DXL XL-320']
Temperatures : [34.0, 33.0, 31.0, 37.0, 37.0, 29.0, 28.0]
Voltages : [0.0, 0.0, 0.0, 11.2, 11.2, 7.3, 7.1]
Hardware errors : [0, 0, 0, 0, 32, 0, 0]


In [64]:
# Ensure the tool is updated an get the current tool in use
Robot.update_tool()
tool = Robot.get_current_tool_id()
print(tool) # ToolID.GRIPPER_1 is the tool we are using for our 
            # robot experiment and it is a small gripper

ToolID.GRIPPER_1


In [66]:
# Move the robot to the default location, this is done to ensure movement is
# based on what is defined from the origin
# Set to the learning mode to falseto prevent external adjustment during trajectory
Robot.set_learning_mode(False)
# adjust the pose to move to the home location
Robot.move_to_home_pose()
# set the leanring mode back to true to allow external adjustment
Robot.set_learning_mode(True)

### Manipulating the Tool values and adjusting tool behaviour
In this section we will evaluate tool behaviour and compare and constrast low level tool APIs over  high level grasp and release APIs

In [67]:
# update the current robot tool
Robot.update_tool()
# This ensures the robot is aware of what tool is currently in use

In [68]:
# get the current tool id
current_tool = Robot.get_current_tool_id()
print(current_tool)

ToolID.GRIPPER_1


In [69]:
# reboot tool
# the tool reboot is used when we want to reset the tool after it gets stuck
Robot.tool_reboot()

In [72]:
# Using high level grasp and release APIs
# GRASP
Robot.grasp_with_tool()
# wait for a few seconds
Robot.wait(3)
# RELEASE
Robot.release_with_tool()
# wait for a few seconds
Robot.wait(3)
# GRASP
Robot.grasp_with_tool()

In [74]:
# Using low level open and close gripper on niryo robot
# The only adjustable parameter we can use for our niryo robot is the speed
gripper_speed = 1000

# the low level gripper API is not working
# GRASP
Robot.close_gripper()
# wait for a few seconds
Robot.wait(3)
# RELEASE
Robot.open_gripper()
# wait for a few seconds
Robot.wait(3)
# GRASP
Robot.close_gripper()

Speed difference across the various speeds is very small and negligible, it would be better to use the high level grasp with tool and release with tool APIs

### Movement, Joints, Poses and Trajectories
In this section we will cover robot movement and pose movement, this will be valuable for us to map the entire scope of location where the robot can move to and map it to pixel images

In [75]:
# check if the robot needs calibration and if it needs calibration perform the calibration
Robot.calibrate_auto()
Robot.wait(2)
Robot.set_learning_mode(False)
Robot.wait(2)
Robot.move_to_home_pose()
Robot.wait(2)
Robot.set_learning_mode(True)

In [76]:
# Joint movement experiments
home_pose = Robot.get_pose()
print(home_pose)

x = 0.1239, y = 0.0002, z = 0.1709
roll = -0.021, pitch = 1.031, yaw = 0.001


In [89]:
# move the joint to a specific joint position
Robot.move_joints(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Robot.wait(1)
Robot.move_joints(0.2, -0.3, 0.1, 0.0, 0.5, -0.8)
Robot.release_with_tool()
Robot.wait(2)
Robot.grasp_with_tool()
Robot.wait(1)
Robot.move_to_home_pose()
Robot.wait(1)
Robot.set_learning_mode(True)

NiryoRobotException: Command KO : An error occured while executing the last command : Goal has been aborted : Command has been aborted due to a collision or a motor not able to follow the given trajectory

In [3]:
# Set learning mode to true
Robot.set_learning_mode(True)

In [87]:
# Get current pose
Robot.move_to_home_pose()
Robot.set_learning_mode(True)
pose_read = Robot.get_pose()
print(pose_read)

NiryoRobotException: Command KO : An error occured while executing the last command : Goal has been aborted : Command has been aborted due to a collision or a motor not able to follow the given trajectory

In [ ]:
# Pose object definitions
# x is the vertical distance of the robot end effector to its base, the lower the value of x, the closer the end effector is to the base of the robot 0.14 is the safest posible value for x, the highest save value for x is 0.4, the z values change depending on teh x values
# y is the distance along the horizontal axis of the robot, negetive values are left of the robot
# z determines how high from the base the robot should go, the higher the value of z the higher the robot arm, the lower the value the lower the robot goes, we discovered that 0.09 is the lowest safe value for the robot, we also found that 0.225 id the safest high value that the robot can go to

# The roll, pitch and yaw are best left as they are



In [86]:
pose_target_obj = pyniryo.PoseObject(
    x=0.225, y=0.15, z=0.07,
    roll=0.1, pitch=1.57, yaw=0.0,
)
Robot.pick_from_pose(pose_target_obj)

In [5]:
place = pyniryo.PoseObject(
    x=0.225, y=-0.15, z=0.07,
    roll=-0.1, pitch=1.57, yaw=0.0,
)
Robot.place_from_pose(place)

In [9]:
# moving to a specific pose
# can move to pose as long as poses are valid and can be computed via move-it
Robot.grasp_with_tool()
Robot.move_pose(place)

In [55]:
# Working with joint values to control robot movement
j1 = 0.0 # this joint is responsible for the robot base joint
j2 = 0.0 # this joint is responsible for the robot shoulder joint
j3 = 0.0 # this joint is responsible for the robot elbow joint
j4 = 0.0 # this is responsible for the post shoulder arm twist
j5 = 0.0 # does not control any joints
j6 = 0.0 # controls the wrist joint

Robot.move_joints([j1, j2, j3, j4, j5, j6])


In [57]:
joint_values = Robot.get_joints()
print(joint_values)

[-0.0021666156231653745, 0.29948321208673506, -1.3291746479090476, -0.00153588974175501, -0.01689478715930511, -0.005061454830783556]
